# Instagram Crawler
crawling for text, comments, and photos 

## 0. ready
1) Make sure to install Selenium, BeautifulSoup, and ChromeDriver. <br>
2) https://ipywidgets.readthedocs.io/en/stable/user_install.html >>tqdm


## 1. import packages

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import sys
from imp import reload

######################################################
from bs4 import BeautifulSoup as bs

import re

from urllib.request import urlopen
import requests
import datetime 
import wget
import html2text

import json
from pandas import json_normalize

import pandas as pd
import numpy as np

from random import *

from tqdm.notebook import tqdm

## 2. login 

In [ ]:
reload(sys)
#sys.setdefaultencoding('utf-8') python3에는 디폴트라 굳이 설정 ㄴㄴ

targetInstagramUrl = 'https://www.instagram.com/accounts/login/?source=auth_switcher'

options = webdriver.ChromeOptions()
#mobile_emulation = { "deviceName": "Nexus 5" }
#options.add_experimental_option("mobileEmulation", mobile_emulation)

login_id = input('put your login id : ')
login_pw = input('put your login password : ')

driver = webdriver.Chrome('chromedriver.exe', options=options)
driver.get(targetInstagramUrl)

#login process ----------------------------------------------------------------- start
#login process ----------------------------------------------------------------- start
time.sleep(2)

driver.find_elements_by_name("username")[0].send_keys(login_id)
driver.find_elements_by_name("password")[0].send_keys(login_pw)

driver.find_element_by_xpath("//form/div[4]/button").submit()

time.sleep(3)

driver.find_element_by_xpath("/html/body/div[4]/div/div/div[3]/button[2]").click()
#login process ----------------------------------------------------------------- end
#login process ----------------------------------------------------------------- end

## 3. search for hashtag

In [ ]:
hashtag = input('What are you searching for? : ')
driver.get('https://www.instagram.com/explore/tags/'+ hashtag)

## 4. Get links of each posts

In [ ]:
def get_post_url(count = 100) : 

    links = []
    last_height = driver.execute_script("return document.body.scrollHeight") # Get scroll height

    while len(links) < count :
        i = 0
        
        ## 1. find links in page sources
        source = driver.page_source
        data = bs(source, 'html.parser')
        body = data.find('body')
        script = body.find('div')
        
        pbar = tqdm(total = count)
        for link in script.findAll('a'):
            if re.match("/p", link.get('href')):
                links.append('https://www.instagram.com'+link.get('href'))
                pbar.update(1)
         
        links = list(set(links))
        
        
        
        ## 2. Scroll down to bottom        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        ## 3. Wait to load page
        time.sleep(uniform(0, 1.0))

        ## 4. Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(uniform(0.5, 1.0))
        
            
        ## 4 - 1. if all pages are already scrolled : break.
        if new_height == last_height:
            break

        ## 4 - 2. else : going on
        else : 
            last_height = new_height
    
    pbar.close()
    links_final = links[0:count]

    return links_final

In [ ]:
#test
a = get_post_url(50)

In [ ]:
len(a)

In [ ]:
a

## Get information from each link of link list

In [ ]:
def info_from_post(link_list) : #get information from each link of link list
    result = pd.DataFrame()
    
    for i in range(len(link_list)) : 
        try:

            page = urlopen(link_list[i]).read()
            data = bs(page, 'html.parser')
            body = data.find('body')
            script = body.find('script')
            raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
            json_data=json.loads(raw)

            posts =json_data['entry_data']['PostPage'][0]['graphql']
            posts = json.dumps(posts)
            posts = json.loads(posts)

            x = pd.DataFrame.from_dict(json_normalize(posts), orient='columns') 
            x.columns =  x.columns.str.replace("shortcode_media.", "")
            result = result.append(x)

        except:
            np.nan
            
    result = result.drop_duplicates(subset = 'shortcode')
    result.index = range(len(result.index))
        
    return result


In [ ]:
res = info_from_post(a)

In [ ]:
res.head()

In [ ]:
res.to_csv('savingname.csv', encoding = 'UTF-8-sig') #encoding for excel